<a href="https://colab.research.google.com/github/kaiquecodes/Analise-do-Enem-2019-no-RN/blob/main/Aplicacao_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Plotly==4.12
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

In [ ]:
import pandas as pd 
import zipfile
import matplotlib

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
# Pacotes para criação de processos para suportar o serviço URL externo do Ngrok
import os.path
import sys, json
import requests
import subprocess

# Pacotes de tratamento de URL
from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

In [ ]:
# Definição da função de Download do Ngrok
def download_ngrok():
    if not os.path.isfile('ngrok'):
        !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
        !unzip -o ngrok-stable-linux-amd64.zip
    pass

In [ ]:
# Criação de um Túnel entre uma aplicação no Colab e uma URL externa no Ngrok
# Função para pegar a referência nesse túnel

Response = namedtuple('Response', ['url', 'error'])

def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

In [ ]:
download_ngrok()

In [ ]:
pd.options.plotting.backend = 'plotly'

In [ ]:
# Geração do programa exemplo_dash_1.py
%%writefile exemplo_dash_1.py
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

microdados_RN= pd.read_csv("microdados_RN.csv")

microdados_RN = microdados_RN.rename({'Q006': 'Renda_Familiar','NO_MUNICIPIO_PROVA': 'Município da Prova','NU_NOTA_MT':'Nota de Matemática','NU_NOTA_CN':'Nota Ciências da Natureza'
,'NU_NOTA_CH':'Nota Ciências Humanas', 'NU_NOTA_LC' : 'Nota Linguagens','NU_NOTA_REDACAO':'Nota da Redação','MEDIA_NOTAS':'Média das Notas'
, 'TIPOS_DE_ESCOLA':'Tipo de Escola', 'NU_IDADE': 'Idade dos Candidatos','Q001': 'Escolaridade_Pai', 'Q002':'Escolaridade da Mãe',}, axis = 1)

microdados_RN = microdados_RN.dropna(subset=['Renda_Familiar','Nota Ciências da Natureza','Nota Ciências Humanas','Nota Linguagens','Nota de Matemática','Nota da Redação'])
col = microdados_RN.loc[:,"Nota Ciências da Natureza":"Nota da Redação"]
microdados_RN['Média das Notas'] = col.mean(axis=1)


#cidades

colunas_selecionadas_cidades = ['Município da Prova','Nota Ciências da Natureza','Nota Ciências Humanas','Nota Linguagens','Nota de Matemática','Nota da Redação','Média das Notas']
cidades = microdados_RN.filter(items=colunas_selecionadas_cidades)
cidades = cidades.dropna()
cidades.value_counts('Município da Prova')

cidades = cidades.groupby('Município da Prova').mean()
fig1 = px.line(cidades, labels=dict(value="Notas", variable="Variável"),color_discrete_sequence=px.colors.qualitative.T10)

#renda

dicionario_renda= {'A':'Nenhuma Renda',
              'B':'998,00','C': '998,01 até 1.497,00', 'D':'1.497,01 até 1.996,00', 
              'E':'1.996,01 até 2.495,00','F':'2.495,01 até 2.994,00','G':'2.994,01 até 3.992,00',
              'H':'3.992,01 até 4.990,00','I':'4.990,01 até 5.988,00','J':'5.988,01 até 6.986,00',
              'K':'6.986,01 até 7.984,00','L':'7.984,01 até 8.982,00','M':'8.982,01 até 9.980,00',
              'N':'9.980,01 até 11.976,00','O':'11.976,01 até 14.970,00','P':'14.970,01 até 19.960,00',
              'Q':'Mais de 19.960,00'}
colunas_selecionadas = ['Renda_Familiar','Média das Notas']
renda = microdados_RN.filter(items=colunas_selecionadas)
renda = renda.dropna()
renda['RENDA'] = [dicionario_renda[resp] for resp in renda.Renda_Familiar]
renda = renda.groupby('RENDA').mean().sort_values(by='Média das Notas')
fig2 = px.area(renda, labels=dict(value="Notas", variable="Variável",),color_discrete_sequence=px.colors.qualitative.Prism)


#Alunos por tipo de escola
colunas_selecionadas_escola = ['TP_ESCOLA','Média das Notas']
dicionario_escola={1.0:'Não respondeu',2.0:'Pública',3.0:'Privada'}
tipo_escola = microdados_RN.filter(items=colunas_selecionadas_escola)
tipo_escola = tipo_escola.dropna()
tipo_escola['Tipo de Escola'] = [dicionario_escola[resp] for resp in tipo_escola.TP_ESCOLA]

fig3 = px.histogram(tipo_escola['Tipo de Escola'],labels=dict(count="Quantidade",value="Tipo de Escola",                                                        
      variable="Variável"),color_discrete_sequence=px.colors.qualitative.Pastel1)

#nota por tipo de escola

tipo_escola = tipo_escola.filter(items=['Tipo de Escola','Média das Notas']).groupby('Tipo de Escola').mean()
fig4 = px.bar(tipo_escola,labels=dict(value="Notas", variable="Variável"),color_discrete_sequence=px.colors.qualitative.Set2)

#idade


idade_hist = microdados_RN['Idade dos Candidatos'].dropna()
fig5 = px.histogram(idade_hist,labels=dict(value="Idade dos Candidatos", count="Idade dos Candidatos", variable="Variável"),
                    color_discrete_sequence=px.colors.qualitative.Safe)

#Notas por Idade

colunas_selecionadas_idade = ['Idade dos Candidatos','Média das Notas']
idade = microdados_RN.filter(items=colunas_selecionadas_idade)
idade = idade.dropna()
idade = idade.groupby('Idade dos Candidatos').mean()

fig6 = px.line(idade,labels=dict(value="Média", variable="Variável"),
               color_discrete_sequence=px.colors.qualitative.Antique)

#notas por escolaridade
dicionario_escolaridade= {'A':'Nunca Estudou',
              'B':'Não completou a 4ª série/5º ano','C': 'Não completou a 8ª série/9º ano', 'D':' Não completou o Ensino Médio.', 
              'E':'Completou Ensino Médio','F':'Completou a Faculdade','G':'Completou a Pós-graduação','H':'Não Sei'}

colunas_selecionadas_escolaridade = ['Escolaridade_Pai','Média das Notas']
escolaridade = microdados_RN.filter(items=colunas_selecionadas_escolaridade)
escolaridade['Escolaridade_Pai'] = [dicionario_escolaridade[resp] for resp in escolaridade.Escolaridade_Pai]
escolaridade = escolaridade.groupby('Escolaridade_Pai').mean().sort_values(by='Média das Notas')

fig7 = px.line(escolaridade, labels=dict(value="Média", variable="Variável"),
               color_discrete_sequence=px.colors.qualitative.D3)

#Quantidade de candidatos por etinia
colunas_selecionadas_etinia = ['TP_COR_RACA','Média das Notas']
dicionario_etinia={0:'Não declarado',1:'Branca',2:'Preta',3:'Parda',4:'Amarela',5:'Indígena'}
etinia = microdados_RN.filter(items=colunas_selecionadas_etinia)
etinia = etinia.dropna()
etinia['etnia'] = [dicionario_etinia[resp] for resp in etinia.TP_COR_RACA]

fig8 = px.histogram(etinia['etnia'], labels=dict(value="Etnia", variable="Variável"),
                    color_discrete_sequence=px.colors.qualitative.Dark2)
#Média dos candidatos por etnia

etinia = etinia.filter(items=['etnia','Média das Notas']).groupby('etnia').mean()
fig9 = px.bar(etinia, labels=dict(value="Média", variable="Variável"))


app.layout = html.Div(children=[
    html.H1(children='Análise de Dados do Enem 2019',style={'textAlign': 'center',}
           ),

    html.H2(children='Análise do desempenho dos estudantes do Rio Grande do Norte no Enem 2019.',
        style={'textAlign': 'center',
        }),
   
    html.Div(children=[
          html.H4(children='Notas por cidades',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''No gráfico abaixo é possível observar a relação entre as notas máximas e mínimas obtidas em cada cidade,
              com isso é possível fazer uma comparação entre as cidades em relação às notas, como qual cidade obteve a maior ou menor nota 
              em cada matéria e, também, perceber qual cidade obteve a melhor ou pior média de matérias. 
              Além disso, pode-se realizar um comparativo da própria cidade, determinando o desempenho  
              da população daquela cidade (que fez o ENEM) em cada matéria.''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
                    dcc.Graph(id='example-graph1', figure=fig1),
             ], 
     ),

     html.Div(children=[
          html.H4(children='Notas por Renda',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''Podemos perceber que existe uma relação quase linear entre as médias das notas e a renda do candidato.
              Quem tem uma renda maior sai com a maior nota. Com isso percebemos que a renda é um fator decisivo na média final. 
              Podemos ver que a diferença entre o mínimo e máximo da média foi de aproximadamente 250 pontos.
''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
             dcc.Graph(id='example-graph2', figure=fig2)
             ],    
     ), 
    
    html.Div(children=[
          html.H4(children='Quantidade de Alunos por Tipo de Escola',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''A partir deste gráfico é possível observar a quantidade de candidatos de cada
              tipo de instituição. Inicialmente percebe-se que a diferença entre a coluna das instituições 
              públicas e privadas é considerável, porém é importante salientar que o número de candidatos 
              que não responderam atrapalha a determinação da maioria.''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
             dcc.Graph(id='example-graph3', figure=fig3),
             ],  
     ),
   html.Div(children=[
          html.H4(children='Notas por Tipo de Escola',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''A partir deste gráfico é possível observar as diferenças das médias de notas dos estudantes 
             de cada tipo de instituição. Ou seja, nela pode-se perceber que a diferença de média entre a rede privada e a pública
              é de 70.7, porém houveram muitos candidatos que não responderam, fazendo com que a média não fique tão real. Além disso, percebe-se que 
              a média dos que não responderam é maior do que os de escola pública. ''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
             dcc.Graph(id='example-graph4', figure=fig4)
             ],    
     ),
   html.Div(children=[
          html.H4(children='Histograma da idade dos candidatos',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''Esse gráfico mostra a quantidade de pessoas que fizeram o enem em relação com a idade. 
             Por exemplo, é possível ver que 8566 pessoas de 20 anos de idade fizeram a prova em 2019. 
             Além disso, percebe-se também que a maioria dos candidatos tinham 18 anos, sendo este o pico mais alto do gráfico. ''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
             dcc.Graph(id='example-graph5', figure=fig5)
             ],    
     ),
   html.Div(children=[
          html.H4(children='Média de notas por faixa etária',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''Neste gráfico são relacionadas a média de notas com a idade do candidato, ou seja, 
             pegou-se as notas de todos os candidatos da mesma idade, fez-se uma média e esta foi relacionada com a idade escolhida. 
             Por exemplo, a maior média obtida (o maior ponto do gráfico) foi dos alunos de 14 anos; porém, observando o gráfico anterior 
             (Histograma da idade dos candidatos) percebe-se que há um número muito pequeno de candidatos nessa faixa etária (apenas 50), 
             o que faz com que as notas boas impactem ainda mais na média destes.''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
             dcc.Graph(id='example-graph6', figure=fig6)
             ],    
     ),
     html.Div(children=[
          html.H4(children='Média de notas por escolaridade do pai',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''Neste gráfico há a relação da média com o grau de escolaridade dos pais. 
             Percebe-se que a média aumenta proporcionalmente com a escolaridade, ou seja, quanto mais diplomas o pai tem, 
             maior a média de notas. Pode-se fazer uma relação deste gráfico com o de “Notas por renda”, pois nele há também um aumento da 
             média proporcional à renda; ou seja, é possível inferir que quanto maior a escolaridade do pai, maior a renda da família e maior
             a média dos candidatos.''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
             dcc.Graph(id='example-graph7', figure=fig7),
             ],    
     ),
     html.Div(children=[
          html.H4(children='Histograma de Candidatos por Raça',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''Este gráfico relaciona a quantidade de candidatos com a sua respectiva raça. 
             Dele pode-se inferir que há uma predominância de candidatos pardos e uma minoridade de indígenas. ''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
             dcc.Graph(id='example-graph8', figure=fig8)
             ],    
     ),

     html.Div(children=[
          html.H4(children='Média das Notas por Raça',style={'textAlign': 'center','font-weight': 'bold'}
             ),
             html.P(children='''Este gráfico relaciona as médias das notas e a raça dos candidatos. 
             Nele é possível observar que não há muita disparidade entre as médias de pessoas de raças diferentes.''',
                    style={'textAlign': 'justify','fontSize':'20px'}),
             dcc.Graph(id='example-graph9', figure=fig9)
             ],    
     ),
    
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
retorno = get_tunnel()
print(retorno)  # Para podermos saber o endereço da URL da Aplicação
!python exemplo_dash_1.py